상추 아닌데 벌레먹은 잎의 사진은 포함했고  
상추가 아닌 것 최대한 제외했는데(놓친 부분 많고)  
아주 건강해보이는데 확인이 어려운 것들은 그냥 두었음

In [ ]:
import matplotlib.pyplot as plt
import os

root_dir = '/home/lab38/images/상추_해충'
file_name = os.listdir(root_dir)
file_dir = [os.path.join(root_dir, x) for x in file_name]
remove_list = []

for f in file_dir :
    plt.figure(figsize=(10,10))
    img = plt.imread(f)
    plt.imshow(img)
    plt.axis('off')
    plt.show()
    
    print('관련없는 사진이면 n')
    while True :
        c = input('y or n : ')
        if c == 'y':
            break
        elif c=='n':
            remove_list.append(f)
            break
        else :
            print('TYPO!!')
    print('next')
    
for f in remove_list :
    if os.path.exists(f):
        os.remove(f)
print('all removed!!')
print(len(os.listdir(root_dir) ,'files left')

In [2]:
import matplotlib.pyplot as plt
import os

root_dir = '/home/lab38/images/상추_해충'
file_name = os.listdir(root_dir)
file_dir = [os.path.join(root_dir, x) for x in file_name]
print(len(file_dir))

226


In [4]:
file_dir[0]

'/home/lab38/images/상추_해충/112.jpg'

## 1. 이미지 리스트 만들기

In [ ]:
img_list = []
grow = []

for i,f in enumerate(file_dir) :
    plt.figure(figsize=(10,10))
    img = plt.imread(f)
    plt.imshow(img)
    plt.axis('off')
    plt.show()
    
    print('pretrainnet에 사용할 이미지이면 y')
    while True :
        c = input('y or n : ')
        if c == 'y':
            img_list.append(f.split('/')[-1])
            grow.append(int(input('애기-0, 다 자라면-1 :')))
            break
        elif c=='n':
            break
        else :
            print('TYPO!!')
            
    
    print(i, 'next')

In [7]:
import pandas as pd
result = pd.DataFrame({'image' : img_list,
                      'grow' : grow})
print(result.__len__())
result.head()

54


,image,grow
0,358.jpg,1
1,235.jpg,1
2,39.jpg,0
3,345.jpg,1
4,97.jpg,1


In [19]:
import albumentations as A

transform = A.Compose([
    A.RandomCrop(width=256, height=256),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.OneOf([
        A.ElasticTransform(p=0.5, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
        A.GridDistortion(p=0.5),
        A.OpticalDistortion(distort_limit=1, shift_limit=0.5, p=1),
    ], p=0.8)
])

In [33]:
import cv2
from tqdm.auto import tqdm

/home/ubuntu/anaconda3/envs/smartpot/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [44]:
root_dir = '/home/lab38/images/상추_해충'
folder_dir = '/home/lab38/상추'

aug_img_list = []
aug_grow = []

for idx, img_name in enumerate(tqdm(result['image'])):
    from_dir = os.path.join(root_dir, img_name)
    
    img = plt.imread(from_dir)
    img = cv2.resize(img, dsize=(320,320))
    plt.imsave(os.path.join(folder_dir, img_name), img)
    grow_label = result.loc[idx, 'grow']
    
    # augmentation
    for i in range(3) :
        aug_img = transform(image=img)
        aug_img = aug_img['image']
        aug_img_name = img_name.split('.')[0]+'_a'+str(i)+'.'+img_name.split('.')[-1]
        
        # labeling
        aug_img_list.append(aug_img_name)
        aug_grow.append(grow_label)
        
        #img save
        plt.imsave(os.path.join(folder_dir, aug_img_name), aug_img)

100%|███████████████████████████████████████████| 54/54 [00:01<00:00, 36.14it/s]


In [50]:
total_img = result['image'].to_list() + aug_img_list
len(total_img)

216

In [53]:
total_grow = result['grow'].to_list() + aug_grow
len(total_grow)

216

In [55]:
aug_result = pd.DataFrame({'image' : total_img,
                      'grow' : total_grow})
print(aug_result.__len__())
aug_result.head()

216


,image,grow
0,358.jpg,1
1,235.jpg,1
2,39.jpg,0
3,345.jpg,1
4,97.jpg,1


In [61]:
aug_result['image'] = aug_result['image'].map(lambda x: os.path.join(folder_dir, x))
aug_result['disease'] = 12
aug_result.head()

,image,grow,disease
0,/home/lab38/상추/358.jpg,1,12
1,/home/lab38/상추/235.jpg,1,12
2,/home/lab38/상추/39.jpg,0,12
3,/home/lab38/상추/345.jpg,1,12
4,/home/lab38/상추/97.jpg,1,12


In [62]:
aug_result['disease-grow'] = list(map(lambda x,y : str(x)+'-'+str(y), aug_result['disease'], aug_result['grow']))
aug_result.head()

,image,grow,disease,disease-grow
0,/home/lab38/상추/358.jpg,1,12,12-1
1,/home/lab38/상추/235.jpg,1,12,12-1
2,/home/lab38/상추/39.jpg,0,12,12-0
3,/home/lab38/상추/345.jpg,1,12,12-1
4,/home/lab38/상추/97.jpg,1,12,12-1


In [68]:
lettu = pd.read_csv('/home/lab38/Multi_proj_6/data/lettuce_smallsizeimg.csv')

In [69]:
lettu.head()

,image,disease,area,grow,points,original,disease-grow
0,/home/lab38/상추/V006_77_0_00_05_03_11_0_b04_202...,0,3,11,"[{'xtl': 2034, 'ytl': 1153, 'xbr': 2436, 'ybr'...",NaN,0-11
1,/home/lab38/상추/V006_77_0_00_05_03_11_0_b04_202...,0,3,11,"[{'xtl': 1513, 'ytl': 676, 'xbr': 2325, 'ybr':...",NaN,0-11
2,/home/lab38/상추/V006_77_0_00_05_03_11_0_b04_202...,0,3,11,"[{'xtl': 1272, 'ytl': 1220, 'xbr': 2229, 'ybr'...",NaN,0-11
3,/home/lab38/상추/V006_77_0_00_05_03_11_0_b04_202...,0,3,11,"[{'xtl': 1723, 'ytl': 1388, 'xbr': 2514, 'ybr'...",NaN,0-11
4,/home/lab38/상추/V006_77_0_00_05_03_11_0_b04_202...,0,3,11,"[{'xtl': 1200, 'ytl': 681, 'xbr': 1816, 'ybr':...",NaN,0-11


In [70]:
lettu['grow'].unique()

array([11, 12])

In [72]:
from sklearn.preprocessing import LabelEncoder
disease_encoder = LabelEncoder()
disease_encoder.fit(lettu['grow'])
lettu['grow'] = disease_encoder.transform(lettu['grow'])
print(lettu['grow'].unique())

[0 1]


In [73]:
new_lettu = pd.concat([aug_result, lettu])
new_lettu.head()

,image,grow,disease,disease-grow,area,points,original
0,/home/lab38/상추/358.jpg,1,12,12-1,NaN,NaN,NaN
1,/home/lab38/상추/235.jpg,1,12,12-1,NaN,NaN,NaN
2,/home/lab38/상추/39.jpg,0,12,12-0,NaN,NaN,NaN
3,/home/lab38/상추/345.jpg,1,12,12-1,NaN,NaN,NaN
4,/home/lab38/상추/97.jpg,1,12,12-1,NaN,NaN,NaN


In [74]:
new_lettu.head(-5)

,image,grow,disease,disease-grow,area,points,original
0,/home/lab38/상추/358.jpg,1,12,12-1,NaN,NaN,NaN
1,/home/lab38/상추/235.jpg,1,12,12-1,NaN,NaN,NaN
2,/home/lab38/상추/39.jpg,0,12,12-0,NaN,NaN,NaN
3,/home/lab38/상추/345.jpg,1,12,12-1,NaN,NaN,NaN
4,/home/lab38/상추/97.jpg,1,12,12-1,NaN,NaN,NaN
...,...,...,...,...,...,...,...
5697,/home/lab38/상추/V006_77_1_10_05_03_12_2_1249e_2...,1,10,10-12,3.0,"[{'xtl': 132, 'ytl': 0, 'xbr': 2805, 'ybr': 34...",V006_77_1_10_05_03_12_2_1249e_20201211_31.jpeg
5698,/home/lab38/상추/V006_77_1_10_05_03_12_1_6432e_2...,1,10,10-12,3.0,"[{'xtl': 8, 'ytl': 33, 'xbr': 3144, 'ybr': 2517}]",V006_77_1_10_05_03_12_1_6432e_20201209_54.jpeg
5699,/home/lab38/상추/V006_77_1_10_05_03_12_2_0517e_2...,1,10,10-12,3.0,"[{'xtl': 1134, 'ytl': 1159, 'xbr': 2655, 'ybr'...",V006_77_1_10_05_03_12_2_0517e_20201210_11.jpeg
5700,/home/lab38/상추/V006_77_1_10_05_03_12_2_6998e_2...,1,10,10-12,3.0,"[{'xtl': 322, 'ytl': 0, 'xbr': 3741, 'ybr': 25...",V006_77_1_10_05_03_12_2_6998e_20201208_23.jpeg


In [75]:
new_lettu['disease-grow'].unique()

array(['12-1', '12-0', '0-11', '9-11', '10-11', '0-12', '9-12', '10-12'],
      dtype=object)

In [77]:
new_lettu['grow'].unique()

array([1, 0])

In [78]:
new_lettu.to_csv('/home/lab38/Multi_proj_6/data/lettu_smallsizeimg_with_pest.csv', index=False)